In [2]:
def wolfe_search(f, df, x, p, alpha0, c1, c2):
    """
    Implements the Wolfe line-search algorithm (Algorithm 8) which returns a
    step length fulfilling the strong Wolfe conditions.

    Arguments:
        f      : objective function f(x)
        df     : gradient function df(x)
        x      : current iterate
        p      : search direction
        alpha0 : initial upper bracket / step-length guess
        c1, c2 : Wolfe condition parameters (0 < c1 < c2 < 1)

    Returns:
        alpha    : step length satisfying the strong Wolfe conditions
        brackets : list of [l, u] brackets recorded at each iteration
    """
    # Scalar wrappers: g(a) = f(x + a*p),  g'(a) = ∇f(x+a*p)ᵀp
    def g(a):
        return f(x + a * p)

    def dg(a):
        return np.inner(df(x + a * p), p)

    l = 0
    u = alpha0
    brackets = [[l, u]]   # record initial bracket (iteration 1)

    # ── Phase 1: expansion ────────────────────────────────────────────
    while True:
        if g(u) > g(0) + c1 * u * dg(0) or g(u) > g(l):
            break
        if abs(dg(u)) < c2 * abs(dg(0)):
            return u, brackets
        if dg(u) > 0:
            break
        else:
            u = u * 2
            brackets.append([l, u])

    # ── Phase 2: bisection ────────────────────────────────────────────
    while True:
        brackets.append([l, u])
        a = (l + u) / 2
        if g(a) > g(0) + c1 * a * dg(0) or g(a) > g(l):
            u = a
        else:
            if abs(dg(a)) < c2 * abs(dg(0)):
                return a, brackets
            if dg(a) < 0:
                l = a
            else:
                u = a

In [3]:
def bfgs(f, df, x0, c1, c2, tol=1e-6, max_iter=1000):
    """
    Implements the BFGS Quasi-Newton algorithm (Algorithm 9) using
    wolfe_search (Algorithm 8) for the line search.

    Arguments:
        f        : objective function f(x)
        df       : gradient function df(x)
        x0       : initial iterate
        c1, c2   : Wolfe condition parameters (0 < c1 < c2 < 1)
        tol      : stopping tolerance on gradient norm
        max_iter : maximum number of iterations

    Returns:
        x        : estimate of local minimum
        iterates : list of iterates [x_0, x_1, ..., x_{k+1}]
    """
    n = len(x0)
    H = np.eye(n)          # H_0 = I_n
    x = x0.copy()
    iterates = [x.copy()]

    for k in range(max_iter):
        grad = df(x)

        # Stopping condition
        if np.linalg.norm(grad) < tol:
            break

        # p_k = -H_k ∇f(x_k)
        p = -H @ grad

        # α_k = wolfe_search(x_k, p_k, 1.0, c1, c2)
        alpha, _ = wolfe_search(f, df, x, p, 1.0, c1, c2)

        # x_{k+1} = x_k + α_k p_k
        x_new = x + alpha * p

        # y_k = ∇f(x_{k+1}) - ∇f(x_k)
        y = df(x_new) - grad

        # s_k = α_k p_k
        s = alpha * p

        # BFGS update of H
        sy = s @ y                          # s_k^T y_k  (scalar)
        Hy = H @ y                          # H_k y_k

        H = (H
             + ((sy + y @ Hy) / sy**2) * np.outer(s, s)
             - (np.outer(Hy, s) + np.outer(s, Hy)) / sy)

        x = x_new
        iterates.append(x.copy())

    return x, iterates
